## Setup

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chess, chess.pgn, chess.svg

In [ ]:
pgn_fn = "lichess_2025-03-07.pgn"
games = []
with open(pgn_fn) as f:
    while True:
        game = chess.pgn.read_game(f)
        if not game:
            break
        else:
            games.append(game)
len(games)

In [ ]:
g = games[0]
dict(g.headers)

## Time

In [ ]:
# Extract date and time information from games
game_times = []
for game in games:
    date_str = game.headers.get("UTCDate", "")
    time_str = game.headers.get("UTCTime", "")
    datetime_str = f"{date_str.replace('.', '-')} {time_str}"
    game_datetime = pd.to_datetime(datetime_str)
    game_times.append(game_datetime)

# Create DataFrame with timestamps
df = pd.DataFrame({'timestamp': game_times})

# Add columns for analysis
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day
df['weekday'] = df['timestamp'].dt.weekday
df['weekday_name'] = df['timestamp'].dt.day_name()

# Plot distribution of games by hour of day
# plt.figure(figsize=(12, 6))
# df['hour'].value_counts().sort_index().plot(kind='bar')
# plt.title('Games Played by Hour of Day')
# plt.xlabel('Hour (UTC)')
# plt.ylabel('Number of Games')
# plt.xticks(rotation=0)
# plt.grid(axis='y', alpha=0.3)
# plt.show()

# Plot games over time
plt.figure(figsize=(14, 6))
time_series = pd.Series(1, index=df['timestamp'])
hourly_counts = time_series.resample('h').count()
hourly_counts.plot()
plt.title('Games Played Over Time')
plt.xlabel('Date/Time')
plt.ylabel('Number of Games')
plt.grid()
plt.show()

# Heatmap of games by weekday and hour
weekday_hour_counts = pd.crosstab(df['weekday_name'], df['hour'])
plt.figure(figsize=(14, 8))
plt.pcolormesh(weekday_hour_counts.columns, range(len(weekday_hour_counts.index)), 
               weekday_hour_counts.values, cmap='viridis')
plt.colorbar(label='Number of Games')
plt.title('Games by Weekday and Hour')
plt.xlabel('Hour of Day (UTC)')
plt.ylabel('Day of Week')
plt.yticks(range(len(weekday_hour_counts.index)), weekday_hour_counts.index)
plt.grid(False)
plt.show()

## Capturing

In [ ]:
# Extract piece captures from the moves
username = ""
pieces = ['P', 'B', 'N', 'Q', 'K', 'R']
captured_pieces = {piece : 0 for piece in pieces}
lost_pieces = {piece : 0 for piece in pieces}

for game in games:
    # Get if user is white or black
    is_white = game.headers['White'].lower() == username.lower()
    board = chess.Board()
    for move in game.mainline_moves():
        if board.is_capture(move):
            # Get the piece that was captured
            to_square = move.to_square
            captured = board.piece_at(to_square)
            if captured:
                piece_name = captured.symbol().upper()
                # If user made the capture
                if (is_white and board.turn == chess.BLACK) or \
                   (not is_white and board.turn == chess.WHITE):
                    captured_pieces[piece_name] += 1
                # If user lost the piece
                else:
                    lost_pieces[piece_name] += 1


plt.figure(figsize=(12, 6))
captured_counts = [captured_pieces[piece] for piece in pieces]
lost_counts = [lost_pieces[piece] for piece in pieces]
x = np.arange(len(captured_counts))
width = 0.35
plt.bar(x - width/2, captured_counts, width, label='Captured by User')
plt.bar(x + width/2, lost_counts, width, label='Captured by Opponent')
plt.title('Piece Capture Analysis')
plt.xlabel('Piece')
plt.ylabel('Count')
plt.xticks(x, pieces, rotation=0)
# plt.yscale('log')
plt.grid(axis='y')
plt.legend()
plt.show()
